<a href="https://colab.research.google.com/github/cclljj/LJ-test/blob/master/LJ_version_of_ipyLeaflet_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ipyleaflet

首先先安裝必要的套件，並載入必要的套件

In [1]:
!pip install -q geopandas
!pip install -q ipyleaflet
!pip install -q osmnx
!pip install -q pyCIOT
!pip install -q geocoder
!pip install -q streamlit


import pandas as pd
import geopandas as gpd
import osmnx
import json
import ipyleaflet
import geocoder
import streamlit
from pyCIOT.data import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 25.2 MB/s 
     |████████████████████████████████| 164 kB 45.9 MB/s 
     |████████████████████████████████| 181 kB 57.5 MB/s 
     |████████████████████████████████| 4.7 MB 66.4 MB/s 
     |████████████████████████████████| 78 kB 8.8 MB/s 
     |████████████████████████████████| 235 kB 77.2 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
    

- 使用空品資料中環保署測站為例，先獲取環保署空品測站的最新資料
- 將獲得的資料由原本的json 轉成 dataframe


In [3]:
epa_station = Air().get_data(src="OBS:EPA")
df = pd.json_normalize(epa_station) 
df['O3'] = 0
for index, row in df.iterrows():
  sensors = row['data']
  for sensor in sensors:
    if sensor['name'] == 'O3':
      df.at[index, 'O3'] =  sensor['values'][0]['value']
df

,name,description,data,properties.city,properties.areaName,properties.township,properties.authority,properties.stationID,properties.englishName,properties.stationName,properties.stationType,location.latitude,location.longitude,location.address,O3
0,空氣品質測站-基隆,空氣品質測站-基隆,"[{'name': 'PM10', 'description': '懸浮微粒 PM10', ...",基隆市,北部空品區,信義區,行政院環境保護署,EPA001,Keelung,基隆,一般測站,25.1292,121.7601,基隆市信義區東信路324號,8
1,空氣品質測站-汐止,空氣品質測站-汐止,"[{'name': 'O3', 'description': '臭氧 O3', 'value...",新北市,北部空品區,汐止區,行政院環境保護署,EPA002,Xizhi,汐止,一般測站,25.0657,121.6408,新北市汐止區樟樹一路137巷26號,11
2,空氣品質測站-萬里,空氣品質測站-萬里,"[{'name': 'NOx', 'description': '氮氧化物 NOx', 'v...",新北市,北部空品區,萬里區,行政院環境保護署,EPA003,Wanli,萬里,一般測站,25.1797,121.6899,新北市萬里區瑪鋉路221號,4
3,空氣品質測站-新店,空氣品質測站-新店,"[{'name': 'AQI', 'description': '空氣品質指標', 'val...",新北市,北部空品區,新店區,行政院環境保護署,EPA004,Xindian,新店,一般測站,24.9772,121.5378,新北市新店區民族路108號,12
4,空氣品質測站-土城,空氣品質測站-土城,"[{'name': 'PM2.5_AVG', 'description': '細懸浮微粒移動...",新北市,北部空品區,土城區,行政院環境保護署,EPA005,Tucheng,土城,一般測站,24.9825,121.4519,新北市土城區學府路1段241號,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,空氣品質測站-關山,空氣品質測站-關山,"[{'name': 'AQI', 'description': '空氣品質指標', 'val...",臺東縣,花東空品區,關山鎮,行政院環境保護署,EPA080,Guanshan,關山,其他測站,23.0451,121.1619,臺東縣關山鎮自強路66號,14
73,空氣品質測站-金門,空氣品質測站-金門,"[{'name': 'WindSpeed', 'description': '風速 Wind...",金門縣,離島空品區,金城鎮,行政院環境保護署,EPA101,Kinmen,金門,一般測站,24.4321,118.3123,金門縣金城鎮民權路32號,35
74,空氣品質測站-馬祖,空氣品質測站-馬祖,"[{'name': 'NOx', 'description': '氮氧化物 NOx', 'v...",連江縣,離島空品區,南竿鄉,行政院環境保護署,EPA102,Matsu,馬祖,一般測站,26.1536,119.9525,連江縣南竿鄉介壽村156-10號,49
75,空氣品質測站-馬公,空氣品質測站-馬公,"[{'name': 'CO_8hr', 'description': '一氧化碳8小時移動平...",澎湖縣,離島空品區,馬公市,行政院環境保護署,EPA104,Magong,馬公,一般測站,23.5690,119.5662,澎湖縣馬公市中正路115號,29


In [4]:
cwb_station = Weather().get_station(src="GENERAL:CWB")
cwb_station[0:2]

df2 = pd.json_normalize(cwb_station) 
df2

,name,description,properties.city,properties.township,properties.authority,properties.stationID,properties.stationName,properties.stationType,location.latitude,location.longitude,location.address
0,局屬氣象站-469020-南沙島,局屬氣象站-469020-南沙島,高雄市,旗津區,中央氣象局,469020,南沙島,局屬氣象站,10.371770,114.356682,None
1,局屬氣象站-CAD050-國一N094K,局屬氣象站-CAD050-國一N094K,新竹市,東區,中央氣象局,CAD050,國一N094K,局屬氣象站,24.799541,121.000695,None
2,局屬氣象站-CAF010-國一N174K,局屬氣象站-CAF010-國一N174K,臺中市,西屯區,中央氣象局,CAF010,國一N174K,局屬氣象站,24.204900,120.655200,None
3,局屬氣象站-467480-嘉義,局屬氣象站-467480-嘉義,嘉義市,西區,中央氣象局,467480,嘉義,局屬氣象站,23.497671,120.424783,None
4,局屬氣象站-CAD010-國一S105K,局屬氣象站-CAD010-國一S105K,新竹縣,寶山鄉,中央氣象局,CAD010,國一S105K,局屬氣象站,24.726316,120.940643,None
...,...,...,...,...,...,...,...,...,...,...,...
94,局屬氣象站-467650-日月潭,局屬氣象站-467650-日月潭,南投縣,魚池鄉,中央氣象局,467650,日月潭,局屬氣象站,23.883096,120.899909,None
95,局屬氣象站-CAA040-國三S016K,局屬氣象站-CAA040-國三S016K,臺北市,南港區,中央氣象局,CAA040,國三S016K,局屬氣象站,25.034846,121.607575,None
96,局屬氣象站-CAJ010-國三N252K,局屬氣象站-CAJ010-國三N252K,雲林縣,林內鄉,中央氣象局,CAJ010,國三N252K,局屬氣象站,23.774120,120.640610,None
97,局屬氣象站-467990-馬祖,局屬氣象站-467990-馬祖,連江縣,南竿鄉,中央氣象局,467990,馬祖,局屬氣象站,26.170981,119.915113,None


轉成 geopanda 資料格式，接著使用 ipyleaflet 秀圖

In [5]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['location.longitude'], df['location.latitude']), crs='epsg:4326')
gdf2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2['location.longitude'], df2['location.latitude']), crs='epsg:4326')
geo_data = ipyleaflet.GeoData(
    geo_dataframe=gdf,
    point_style={'radius': 5, 'color': 'black', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name="EPA stations",
)
geo_data2 = ipyleaflet.GeoData(
    geo_dataframe=gdf2,
    point_style={'radius': 5, 'color': 'black', 'fillOpacity': 0.8, 'fillColor': 'red', 'weight': 3},
    name="CWB stations",
)


m1 = ipyleaflet.Map(center=(23.8, 121), zoom =8, basemap= ipyleaflet.basemaps.OpenStreetMap.Mapnik)
m1.add_layer(geo_data)
m1.add_layer(geo_data2)
m1.add_control(ipyleaflet.LayersControl(position='topright'))
m1.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
m1.layout.height="800px"
m1

Map(center=[23.8, 121], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [6]:
m2 = ipyleaflet.Map(center=(23.8, 121), zoom =8, basemap= ipyleaflet.basemaps.OpenStreetMap.Mapnik)
m2.add_control(ipyleaflet.LayersControl(position='topright'))
m2.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
m2.layout.height="800px"

markers = []
for index, row in df.iterrows():
  marker = ipyleaflet.Marker(location=(row['location.latitude'],row['location.longitude']), draggable=False, title=row['name'], alt=row['name'])
  markers.append(marker)
marker_cluster = ipyleaflet.MarkerCluster(markers=markers)
m2.add_layer(marker_cluster)

m2

Map(center=[23.8, 121], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

增加不同的底圖圖層，完整的底圖圖層清單在這邊：https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html

In [7]:
m3 = ipyleaflet.Map(center=(23.8, 121), zoom =8, basemap= ipyleaflet.basemaps.OpenStreetMap.Mapnik)
m3.add_layer(geo_data)
m3.add_control(ipyleaflet.LayersControl(position='topright'))
m3.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
m3.layout.height="800px"

layers = list(ipyleaflet.basemaps.keys())
print(len(layers),layers)

layer = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.Stamen.Terrain)
layer.name = "Stamen.Terrain Layer"
layer.base = False
m3.add_layer(layer)
m3

39 ['OpenStreetMap', 'MapTilesAPI', 'OpenSeaMap', 'OPNVKarte', 'OpenTopoMap', 'OpenRailwayMap', 'OpenFireMap', 'SafeCast', 'Stadia', 'Thunderforest', 'CyclOSM', 'Jawg', 'MapBox', 'MapTiler', 'Stamen', 'TomTom', 'Esri', 'OpenWeatherMap', 'HERE', 'HEREv3', 'FreeMapSK', 'MtbMap', 'CartoDB', 'HikeBike', 'BasemapAT', 'nlmaps', 'NASAGIBS', 'NLS', 'JusticeMap', 'GeoportailFrance', 'OneMapSG', 'USGS', 'WaymarkedTrails', 'OpenAIP', 'OpenSnowMap', 'AzureMaps', 'SwissFederalGeoportal', 'Gaode', 'Strava']


Map(center=[23.8, 121], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

save to html

In [ ]:
m3.save("test.html", title = "Hello World!")


# Add OSM data of place(s) by name or ID to the map.

https://pygis.io/docs/d_access_osm.html

可疊加上 OSM 的屬性資料，完整的資料可見 OSM 網站說明：https://wiki.openstreetmap.org/wiki/Map_features


In [8]:
city_name = "Taichung, Taiwan"
city_area = osmnx.geocode_to_gdf(city_name)
geo_city_data = ipyleaflet.GeoData(geo_dataframe = city_area,
                              name = city_name)

m4 = ipyleaflet.Map(center=(24.245, 121), zoom =10, basemap= ipyleaflet.basemaps.OpenStreetMap.Mapnik)
m4.add_layer(geo_data)
m4.add_layer(geo_city_data)
m4.add_control(ipyleaflet.LayersControl(position='topright'))
m4.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
m4

Map(center=[24.245, 121], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [9]:
city_name = "Taichung, Taiwan"
city_schools = osmnx.geometries_from_place(city_name ,tags={"amenity": "school"})
city_schools[0:3]

/usr/local/lib/python3.7/dist-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/usr/local/lib/python3.7/dist-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


barrier                    geometry addr:city  \
element_type osmid                                                      
node         2673110823     NaN  POINT (120.65328 24.15390)       臺中市   
             3010504951     NaN  POINT (120.68592 24.11657)       NaN   
             3010518119     NaN  POINT (120.68590 24.11627)       NaN   

                        addr:district addr:floor              addr:full  \
element_type osmid                                                        
node         2673110823            西區          7  40352臺中市西區東興路三段246號7樓   
             3010504951           NaN        NaN                    NaN   
             3010518119           NaN        NaN                    NaN   

                        addr:housenumber addr:postcode addr:street amenity  \
element_type osmid                                                           
node         2673110823              246         40352       東興路三段  school   
             3010504951              NaN           NaN         NaN  school   
             3010518119              NaN           NaN         NaN  school   

                         ... old_name:en toilets:wheelchair alt_name:ja  \
element_type osmid       ...                                              
node         2673110823  ...         NaN                NaN         NaN   
             3010504951  ...         NaN                NaN         NaN   
             3010518119  ...         NaN                NaN         NaN   

                        name:ja ways type denomination polling_station  \
element_type osmid                                                       
node         2673110823     NaN  NaN  NaN          NaN             NaN   
             3010504951     NaN  NaN  NaN          NaN             NaN   
             3010518119     NaN  NaN  NaN          NaN             NaN   

                        contact:instagram contact:youtube  
element_type osmid                                         
node         2673110823               NaN             NaN  
             3010504951               NaN             NaN  
             3010518119               NaN             NaN  

[3 rows x 94 columns]

In [10]:
city_schools = city_schools.loc[:,city_schools.columns.str.contains('addr:|geometry')]
city_schools = city_schools.loc[city_schools.geometry.type=='Polygon']
city_schools[0:3]

geometry  \
element_type osmid                                                         
way          24304770  POLYGON ((120.62678 24.35149, 120.62601 24.350...   
             24304944  POLYGON ((120.62512 24.34892, 120.62387 24.349...   
             30879359  POLYGON ((120.68521 24.14961, 120.68619 24.149...   

                      addr:city addr:district addr:floor           addr:full  \
element_type osmid                                                             
way          24304770       臺中市           大甲區        NaN  43749臺中市大甲區育德路233號   
             24304944       臺中市           大甲區        NaN  43747臺中市大甲區育德路113號   
             30879359       臺中市            北區        NaN     40403臺中市北區育才街2號   

                      addr:housenumber addr:postcode addr:street addr:country  \
element_type osmid                                                              
way          24304770              233         43749         育德路           TW   
             24304944              113         43747         育德路           TW   
             30879359                2         40403         育才街           TW   

                      addr:village addr:place addr:full:en addr:housename  \
element_type osmid                                                          
way          24304770          NaN        NaN          NaN            NaN   
             24304944          NaN        NaN          NaN            NaN   
             30879359          NaN        新北里          NaN            NaN   

                      addr:TW:dataset addr:hamlet addr:neighbourhood  
element_type osmid                                                    
way          24304770             NaN         NaN                NaN  
             24304944             NaN         NaN                NaN  
             30879359             NaN         NaN                NaN

In [11]:
city_worships = osmnx.geometries_from_place(city_name,tags={"amenity": "place_of_worship"})
city_worships = city_worships.loc[:,city_worships.columns.str.contains('addr:|geometry')]
city_worships = city_worships.loc[city_worships.geometry.type=='Polygon']
city_worships[0:3]

geometry  \
element_type osmid                                                          
way          113836262  POLYGON ((120.63385 24.31670, 120.63438 24.316...   
             114388590  POLYGON ((120.61663 24.33276, 120.61700 24.332...   
             183474386  POLYGON ((120.62346 24.34541, 120.62350 24.345...   

                       addr:city addr:district           addr:full  \
element_type osmid                                                   
way          113836262       NaN           NaN                 NaN   
             114388590       臺中市           外埔區  43859臺中市外埔區山美路813號   
             183474386       臺中市           大甲區  43741臺中市大甲區順天路158號   

                       addr:housenumber addr:postcode addr:street  \
element_type osmid                                                  
way          113836262              NaN           NaN         NaN   
             114388590              813         43859         山美路   
             183474386              158         43741         順天路   

                       addr:country addr:village addr:place addr:floor  \
element_type osmid                                                       
way          113836262          NaN          NaN        NaN        NaN   
             114388590          NaN          NaN        NaN        NaN   
             183474386           TW          NaN        NaN        NaN   

                       addr:TW:dataset addr:hamlet addr:neighbourhood  
element_type osmid                                                     
way          113836262             NaN         NaN                NaN  
             114388590             NaN         NaN                NaN  
             183474386             NaN         NaN                NaN

In [12]:
geo_data_school = ipyleaflet.GeoData(geo_dataframe = city_schools,
                            style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                            name = city_name + ": Schools")
geo_data_worship = ipyleaflet.GeoData(geo_dataframe = city_worships,
                            style={'color': 'black', 'radius':8, 'fillColor': '#cc6633', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                            name = city_name + ": Worships")
m5 = ipyleaflet.Map(center=(24.245, 121), zoom =10, basemap= ipyleaflet.basemaps.OpenStreetMap.Mapnik)
m5.add_layer(geo_data)
m5.add_layer(geo_city_data)
m5.add_layer(geo_data_school)
m5.add_layer(geo_data_worship)
m5.add_control(ipyleaflet.LayersControl(position='topright'))
m5.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
m5

/usr/local/lib/python3.7/dist-packages/ipyleaflet/leaflet.py:2393: DeprecationWarning: add_layer is deprecated, use add instead
  warnings.warn("add_layer is deprecated, use add instead", DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/ipyleaflet/leaflet.py:2466: DeprecationWarning: add_control is deprecated, use add instead
  warnings.warn("add_control is deprecated, use add instead", DeprecationWarning)


Map(center=[24.245, 121], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

# add heatmap for O3

In [13]:
m6 = ipyleaflet.Map(center=(23.8, 121), zoom =8, basemap= ipyleaflet.basemaps.OpenStreetMap.Mapnik)
m6.add_layer(geo_data)
m6.add_control(ipyleaflet.LayersControl(position='topright'))
m6.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
m6.layout.height="800px"

locations = []
for index, row in df.iterrows():
  item = [row['location.latitude'], row['location.longitude'], row['O3']]
  locations.append(item)
heatmap = ipyleaflet.Heatmap(
    locations= locations,
    radius=100,
    name = "O3 Heatmap"
)

m6.add_layer(heatmap);
m6

Map(center=[23.8, 121], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

# But......Streamlit is not working with ipyleaflet :<

# Streamlit


In [17]:
%%writefile app.py
import streamlit as st
import ipyleaflet
import json
import pandas as pd
import geopandas as gpd
from pyCIOT.data import *

contnet = """
Hello World!
"""
st.title('Streamlit Demo')
st.write("## ipyleaflet Example")
st.markdown(contnet)

epa_station = Air().get_data(src="OBS:EPA")
cwb_station = Weather().get_station(src="GENERAL:CWB")
df = pd.json_normalize(epa_station) 
df2 = pd.json_normalize(cwb_station) 
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['location.longitude'], df['location.latitude']), crs='epsg:4326')
gdf2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2['location.longitude'], df2['location.latitude']), crs='epsg:4326')

geo_data = ipyleaflet.GeoData(
    geo_dataframe=gdf,
    point_style={'radius': 5, 'color': 'black', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name="EPA stations",
)
geo_data2 = ipyleaflet.GeoData(
    geo_dataframe=gdf2,
    point_style={'radius': 5, 'color': 'black', 'fillOpacity': 0.8, 'fillColor': 'red', 'weight': 3},
    name="CWB stations",
)


with st.expander("See source code"):
  with st.echo():
    m = ipyleaflet.Map(center=(23.8, 121), zoom =8, basemap= ipyleaflet.basemaps.OpenStreetMap.Mapnik)
    m.add_layer(geo_data)
    m.add_layer(geo_data2)
    m.add_control(ipyleaflet.LayersControl(position='topright'))
    m.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
    m.layout.height="800px"
        
m.to_streamlit()

Overwriting app.py


In [18]:
!streamlit run app.py & npx localtunnel --port 8501

2022-09-27 15:40:15.271 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.528s
your url is: https://easy-yaks-retire-35-245-248-4.loca.lt



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.245.248.4:8501

2022-09-27 15:40:53.950 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 562, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 43, in <module>
    m.to_streamlit()
AttributeError: 'Map' object has no attribute 'to_streamlit'
  Stopping...
^C


# Reference

*   https://coderzcolumn.com/tutorials/data-science/ipyleaflet-interactive-maps-in-python-based-on-leafletjs
*   List item

